In [105]:
import time
import codecs
import ephem
from ephem import degree
import datetime
from datetime import timedelta
import urllib.request
import csv

In [106]:
# Load the Space Catalogue as a dictionary
satcat = "https://celestrak.com/pub/satcat.txt"
sat_catalogue = urllib.request.urlopen(satcat).read().decode('utf-8').strip().split("\n")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue

HTTPError: HTTP Error 403: Forbidden

In [146]:
import json
import csv
import datetime
from datetime import timedelta
import ephem

# Open the local TLE data file
local_tle_file = "update_tle.tsv"
output_file = "output/data.json"

# Read the local TLE data file
with open(local_tle_file, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter='\t')
    sheetValues = list(reader)


In [151]:
data_list = []

for i in range(1, len(sheetValues)):
    row = sheetValues[i]
    tle_line0 = row[0]
    tle_line1 = row[1]
    norad_id = row[2].replace('n_', '')
    possible_satellite_reentry_date = sat_dict[norad_id][75:85] + " " + "00:00:00"
    
    try:
        # Read TLE
        name = "sat_data"
        tle_rec = ephem.readtle(name, tle_line0, tle_line1)
        tle_rec.compute(possible_satellite_reentry_date)
    
    
        # Set reentry threshold to 120 km (120000 meters)
        reentry_threshold = 120000  # official entry interface by ESA and NASA
    
        # Check if the initial elevation is above or below the threshold
        if int(tle_rec.elevation) >= reentry_threshold:

            d = datetime.datetime.strptime(possible_satellite_reentry_date, '%Y-%m-%d %H:%M:%S')

            # Check whether the debris reenters after max two days 
            for i in range(0, 48):  # 48 hours
                parseHour = d + timedelta(hours=i)
                tle_rec.compute(parseHour)

                if int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry   

                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(parseHour),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    
                    # Add further detailed tracking points
                    for offset_minutes in [2, 6]:
                        nextMinutes = parseHour + timedelta(minutes=offset_minutes)
                        tle_rec.compute(nextMinutes)
                        data_list.append({
                            "tle_0": tle_line0,
                            "tle_1": tle_line1,
                            "lat": tle_rec.sublat / ephem.degree,
                            "lon": tle_rec.sublong / ephem.degree,
                            "norad_cat_num": norad_id,
                            "satellite_name": sat_dict[norad_id][23:47].strip(),
                            "ownership": sat_dict[norad_id][49:54].strip(),
                            "launch_date": sat_dict[norad_id][56:66].strip(),
                            "satellite_decay": sat_dict[norad_id][75:85].strip(),
                            "satellite_decay_hour": str(nextMinutes),
                            "rcs": sat_dict[norad_id][119:127].strip()
                        })
                    break
        
            # second attempt
            larger_reentry_threshold = 220000
            
            for i in range(0, 48):  # 48 hours
                parseHour = d + timedelta(hours=i)
                tle_rec.compute(parseHour)

                if int(tle_rec.elevation) <= larger_reentry_threshold:  # threshold of reentry   

                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(parseHour),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    
                    # Add further detailed tracking points
                    for offset_minutes in [2, 6]:
                        nextMinutes = parseHour + timedelta(minutes=offset_minutes)
                        tle_rec.compute(nextMinutes)
                        data_list.append({
                            "tle_0": tle_line0,
                            "tle_1": tle_line1,
                            "lat": tle_rec.sublat / ephem.degree,
                            "lon": tle_rec.sublong / ephem.degree,
                            "norad_cat_num": norad_id,
                            "satellite_name": sat_dict[norad_id][23:47].strip(),
                            "ownership": sat_dict[norad_id][49:54].strip(),
                            "launch_date": sat_dict[norad_id][56:66].strip(),
                            "satellite_decay": sat_dict[norad_id][75:85].strip(),
                            "satellite_decay_hour": str(nextMinutes),
                            "rcs": sat_dict[norad_id][119:127].strip()
                        })
                    break
                        
        elif int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry

            for offset_minutes in [0, 2, 6]:
                nextMinutes = d + timedelta(minutes=offset_minutes)
                tle_rec.compute(nextMinutes)
                data_list.append({
                    "tle_0": tle_line0,
                    "tle_1": tle_line1,
                    "lat": tle_rec.sublat / ephem.degree,
                    "lon": tle_rec.sublong / ephem.degree,
                    "norad_cat_num": norad_id,
                    "satellite_name": sat_dict[norad_id][23:47].strip(),
                    "ownership": sat_dict[norad_id][49:54].strip(),
                    "launch_date": sat_dict[norad_id][56:66].strip(),
                    "satellite_decay": sat_dict[norad_id][75:85].strip(),
                    "satellite_decay_hour": str(nextMinutes),
                    "rcs": sat_dict[norad_id][119:127].strip()
                })
    except Exception as e:
        print(norad_id, e)
        continue

print("\nFinished parsing!")


00099 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 3856 days from the epoch
00161 cannot compute the body's position at 1981/1/8 00:00:00
00356 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 2973 days from the epoch
00399 line does not conform to tle format
00714 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 1393 days from the epoch
00722 line does not conform to tle format
00935 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 602 days from the epoch
01347 line does not conform to tle format
01438 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 3170 days from the epoch
01656 TLE elements are valid for a few weeks around their epoch, but you are asking about a date 768 days from the epoch
01663 line does not conform to tle format
01718 TLE elements are valid for a few w

In [152]:
# Write data to minified JSON file
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, separators=(',', ':'))
